## Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from PIL import Image
import os
from os.path import splitext
from os import listdir
import random
import numpy as np
from tqdm.notebook import tqdm
import math
from skimage.metrics import peak_signal_noise_ratio, normalized_root_mse, structural_similarity

import pytorch_ssim
np.set_printoptions(suppress=True)

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Dataset Definition

In [2]:
class BasicDataset(Dataset):

    def __init__(self, imgs_dir, masks_dir,  filenames, train):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.train = train

        if(self.train):
            self.transforms = train_transforms
        else:
            self.transforms = test_transforms
        
        self.ids = filenames
        self.gt = np.loadtxt(masks_dir + '/test.txt', delimiter=', ')
        self.gt = torch.from_numpy(self.gt)
        self.masks = self.gt[:, :7]
        self.cameras = self.gt[:, 7:9]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, i):
        idx = self.ids[i]
        img_file = glob(self.imgs_dir + idx + '.png')
        mask_id = int(idx.split('_')[1])-1
        mask = self.masks[mask_id, :]

        #assert len(img_file) == 1,             f'Either no image or multiple images found for the ID {idx}: {img_file}'
        #assert len(mask) == 10,             f'Either no mask or multiple masks found for the ID {idx}: {mask}'
        
        img = Image.open(img_file[0])
        
        if(img.mode == 'P'):
            img = img.convert('RGB')
            img.save(self.imgs_dir + idx + '.png')

        return {'image': (self.transforms(img)).float(), 'mask': (mask).float(), 'label': mask_id, 'camera' : self.cameras[mask_id]}
    
class TestDataset(Dataset):

    def __init__(self, gt_path, pred_path):
        self.gt_path = gt_path
        self.pred_path = pred_path

        self.gt_filenames = glob(self.gt_path + 'gt/*.png')
        self.pred_filenames = glob(self.pred_path + 'pred/*.png')

        #assert len(self.gt_filenames) == self.pred_filenames

    def __len__(self):
        return len(self.pred_filenames)

    def __getitem__(self, i):
        gt_image = self.gt_path + 'gt/' + os.path.basename(self.pred_filenames[i])
        pred_image = self.pred_path + 'pred/' + os.path.basename(self.pred_filenames[i])

        img1 = Image.open(gt_image).convert('RGB')
        img2 = Image.open(pred_image).convert('RGB')

        return {'gt': transforms.ToTensor()(img1), 'pred': transforms.ToTensor()(img2)}

## Utils

In [3]:
test_transforms = transforms.Compose([
                           #transforms.Resize(90),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       ])

def loadModel(path, filename, device):
    checkpoint = torch.load(path+'/'+filename, map_location=device)
    epoch = checkpoint['epoch']
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']
    lr_sched = checkpoint['lr_sched']

    return epoch, model, optimizer, lr_sched

def getX(radius, theta, phi):
    return radius * math.sin(phi*0.01745) * math.cos(theta*0.01745);


def getY(radius, theta, phi):
    return radius * math.sin(phi*0.01745) * math.sin(theta*0.01745);


def getZ(radius, theta, phi):
    return radius * math.cos(phi*0.01745);

def getLoss(pred, target):
    loss_Kd = ( (target[:, 0:3] - pred[:, 0:3] )**2).sum()
    factor = torch.unsqueeze((1-torch.pow(target[:,6], 1/3)),dim=1)
    loss_Ks = ( (target[:, 3:6] - pred[:, 3:6] )**2 * factor ).sum()
    loss_r = ( (target[:, 6] - pred[:, 6] )**2).sum()
    return loss_Kd + loss_Ks + (loss_r*3)

def test(model, device, iterator):
    epoch_loss = 0
    global_step = 0
    
    model.eval()
    
    prediction_list = []
    label_list = []
    camera_list = []
   
    with torch.no_grad():
        for batch in iterator:

            x = batch['image']
            y = batch['mask']
            labels =  batch['label']
            cameras = batch['camera']
            
            x = x.to(device)
            y = y.to(device)

            fx = model(x)

            prediction_list.append(torch.squeeze(fx).cpu())
            label_list.append(torch.squeeze(labels))
            camera_list.append(torch.squeeze(cameras))
            loss = getLoss(fx, y)

            epoch_loss += loss.item()
            
            global_step += 1
            
    return epoch_loss / len(iterator), ((torch.stack(prediction_list)).view(-1,7)).numpy(), np.squeeze(((torch.stack(label_list)).view(-1,1)).numpy()), np.squeeze(((torch.stack(camera_list)).view(-1,2)).numpy())

In [6]:
##################################################################################################

dir_img = 'maps-gt/'
dir_mask = 'masks/'
architecture = 'resnext101_32x8d'
model_folder = './'
save_model_folder = model_folder + ''
checkpoint_file = 'efficient_44.pth'
in_feat, out_feat = 2048, 7

##################################################################################################

batch_size = 300
test_file_names = np.array([splitext(file)[0] for file in listdir(dir_img) if not file.startswith('.')])
test_dataset = BasicDataset(dir_img, dir_mask, test_file_names, train=False)
test_iterator = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=32, pin_memory=True)
#torch.save(test_iterator, './test_loader.pth')

device = torch.device('cuda')
# model = torch.hub.load('pytorch/vision:v0.5.0', architecture, pretrained=False)
# model.fc = nn.Sequential(
#             nn.Linear(in_features=in_feat, out_features=out_feat, bias=True),
#             nn.Sigmoid()
#             )

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b0')
model._fc = nn.Sequential(
            nn.Linear(in_features=1280, out_features=7, bias=True),
            nn.Sigmoid()
            )

model.to(device)
epoch, saved_model, optimizer, lr_sched = loadModel(save_model_folder, checkpoint_file, device)
model.load_state_dict(saved_model)

<All keys matched successfully>

## Step 1 : Predicting properties from test images

In [7]:
test_loss, entire_predict, mat_labels, camera_list = test(model, device, test_iterator)
result = np.concatenate((entire_predict, np.expand_dims(mat_labels,axis=1), camera_list), axis=1)
np.savetxt(save_model_folder+'predictions_with_ids.txt', result, fmt='%.3f', delimiter =', ')

## Step 2 : Using predicted properties to re-render images

In [8]:
##################################################################################################

prediction_set = np.loadtxt(save_model_folder+'predictions_with_ids.txt', delimiter=', ')
GT_PATH = 'masks/'
ground_truth =  np.loadtxt(GT_PATH+'test.txt', delimiter=', ')[:,:9]

##################################################################################################

for counter in tqdm(range(len(prediction_set))):
    
    materials = prediction_set[counter][:7]
    label = int(prediction_set[counter][7])
    camera = ground_truth[label][7:9]
    mf = open("utils/materials.pbrt", "w+")
    mf.write('MakeNamedMaterial \"custommaterial\" \"string type\" \"uber\"\n' + '\n\"rgb Kd\" [' + str(materials[0]) + ' ' + str(materials[1]) + ' ' + str(materials[2]) + ']'+ '\n\"rgb Ks\" [' + str(materials[3]) + ' ' + str(materials[4]) + ' ' + str(materials[5]) + '] ' + '\n\"float roughness\" [' + str(materials[6]) + ']\n')
    mf.close()

    directory = model_folder+ 'efficient-pred/'

    cam_pos = getX(2, camera[1], camera[0]), getY(2, camera[1], camera[0]), getZ(2, camera[1], camera[0])

    cf = open("utils/camera.pbrt", "w+")
    ff = open("utils/film.pbrt", "w+")
    cf.write("LookAt ")
    cf.write("\n%f %f %f\n0 0 0\n0 1 0" % (cam_pos[0], cam_pos[2], cam_pos[1]))
    ff.write("Film \"image\" \"string filename\" \"%s_%s_%s.png\" \"integer xresolution\" [128] \"integer yresolution\" [128]" %  (str(directory) +"/"+ str(label+1), str(int(camera[0])), str(int(camera[1])) ))
    cf.close()
    ff.close()
    os.system(r"/home/farhan/Masters_Thesis/pbrt-v3/cmake-build-release/pbrt --quiet --nthreads=64 utils/scene.pbrt")
    #os.rename("/home/farhan/Farhan_Thesis_Codes/pbrt_experiments/kitchen/materials_props.txt", "materials_phi_%s_theta_%s.txt" %  (str(cam_pos[3]), str(cam_pos[4])))

## SSIM

In [21]:
# device = torch.device('cuda')
# sum_ssim = 0
# sum_mse = 0
# sum_psnr = 0

# gt_dir = '/home/farhan/Farhan_Thesis_Codes/pbrt_experiments/test-images/'
# pred_dir = '/home/farhan/Farhan_Thesis_Codes/new_codes/resnet18-cedar/'
# t_data = TestDataset(gt_dir, pred_dir)
# train_iterator = DataLoader(t_data, batch_size=16, shuffle=True, num_workers=32, pin_memory=True)

# for batch in train_iterator:
#     gt_img = batch['gt']
#     target_img = batch['pred']

#     gt_img = gt_img.to(device)
#     target_img = target_img.to(device)
    
#     sum_ssim += pytorch_ssim.ssim(gt_img, target_img)
#     sum_mse += ((gt_img-target_img)**2).mean()

In [9]:
gt_dir = 'new-gt/'
pred_dir = './'

sum_psnr = 0
sum_ssim = 0
sum_nrmse = 0

pred_images = glob(pred_dir+'efficient-pred/*.png')

for i in tqdm(range(len(pred_images))):
    file = pred_images[i]
    base_name = os.path.basename(file)
    img1 = np.array(Image.open(gt_dir + base_name).convert('RGB'))
    img2 = np.array(Image.open(pred_dir+'efficient-pred/'+base_name).convert('RGB'))
    sum_psnr += peak_signal_noise_ratio(img1, img2)
    sum_ssim += structural_similarity(img1, img2, multichannel=True)
    sum_nrmse += normalized_root_mse(img1, img2)

#### Resnet18

In [9]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(45.490021732690025, 0.03538263390718832, 0.003910770852089129)

#### Resnet34

In [22]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(46.36415205005039, 0.034913718536975016, 0.004013628354597731)

#### Resnet50

In [23]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(47.255119025251325, 0.03253375369412408, 0.003977283175405244)

#### ResNext50

In [17]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(47.43280293066334, 0.030880006816994387, 0.0037543722624024944)

#### ResNext101

In [28]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(47.583155822177254, 0.030717176250682005, 0.003572391948516218)

#### Alexnet

In [9]:
sum_psnr/30000, sum_nrmse/30000, sum_ssim/30000, 1/(sum_ssim/30000) -1

(41.6840961704434,
 0.05638451460035347,
 0.9932812720944176,
 0.00676417455391598)

#### WideResnet50

In [27]:
sum_psnr/30000, sum_nrmse/30000, sum_ssim/30000, 1/(sum_ssim/30000) -1

(48.052561074743124,
 0.0287378758416173,
 0.996754223762731,
 0.0032563456064587104)

#### EfficientNet-B0

In [13]:
sum_psnr/30000, sum_nrmse/30000, 1/(sum_ssim/30000)-1

(30.710896729251537, 0.18382442893359596, 0.029410542354880276)